In [4]:
# Import required modules
import pandas as pd

In [5]:
# Read everything as a text file
with open("D:/01_WORK/03_Working_Files/Python/Prosper_VLP_TPD_Reader/vlp_sample.tpd", 'r') as f:
    lines = f.readlines()

In [16]:
# Identify variables included in tpd file
calcVarsHeader = "# Number of Calculated Values (columns)\n"
calcVarsHeaderIndex = lines.index(calcVarsHeader)
calcVarsNum = int(lines[calcVarsHeaderIndex + 1])
calcVarsNames = lines[calcVarsHeaderIndex+2:calcVarsHeaderIndex+calcVarsNum+2]


In [1]:
lines[129]

NameError: name 'lines' is not defined